In [ ]:
import networkx as nx
import numpy as np

def generate_regular_graph():
    # 这里简单以正则图为例, 鼓励同学们尝试在其他类型的图(具体可查看如下的nx文档)上测试算法性能
    # nx文档 https://networkx.org/documentation/stable/reference/generators.html
    graph = nx.random_graphs.random_regular_graph(d=99, n=200, seed=2023)
    return graph, len(graph.nodes), len(graph.edges)

def generate_erdos_renyi_graph():
    graph = nx.random_graphs.erdos_renyi_graph(n=180, p=0.52, seed=2023)
    return graph, len(graph.nodes), len(graph.edges)

data_path = 'regular'

if data_path == 'regular':
    graph, n_nodes, n_edges = generate_regular_graph()
elif data_path == 'ER':
    graph, n_nodes, n_edges = generate_erdos_renyi_graph()
    
print(n_nodes, n_edges)
def get_fitness(graph, x, threshold=0):
    # 获得Cuts值需要将图分为两部分, 这里默认以0为阈值把解分成两块.
    g1 = np.where(x == 0)[0]
    g2 = np.where(x == 1)[0]
    return -nx.cut_size(graph, g1, g2) / n_edges

k = 8 
n = n_nodes

In [ ]:
def crossover(individual_a, individual_b, n):
    l = len(individual_a)
    offspring_a = np.zeros((n))
    offspring_b = np.zeros((n))

    m = np.arange(l) < np.random.randint(l + 1)
    offspring_a = np.where(m, individual_a, individual_b)
    offspring_b = np.where(~m, individual_a, individual_b)

    return [offspring_a, offspring_b]

def mutation(individual):
    p = k / n
    m = np.random.choice((0,1), p=(1-p, p), size=n)
    flip = 1 - individual
    individual = np.where(m, flip, individual)
    return np.array(individual)

# 快速非支配解排序方法
def fast_non_dominated_sort(values1, values2):
    length = len(values1)
    S = [[] for _ in range(length)]
    front = [[]]
    n = [0 for _ in range(length)]
    rank = [0 for _ in range(length)]

    for p in range(length):
        S[p], n[p] = [], 0 # the set of solutions dominated by p, the number of solutions dominating p
        for q in range(length):
            # if p dominates q, add q to S[p]
            if (values1[p] < values1[q] and values2[p] < values2[q]) or \
                (values1[p] <= values1[q] and values2[p] < values2[q]) or \
                    (values1[p] < values1[q] and values2[p] <= values2[q]):
                if q not in S[p]:
                    S[p].append(q)
            # if q dominates p, increase n[p]
            elif (values1[q] < values1[p] and values2[q] < values2[p]) or\
                (values1[q] <= values1[p] and values2[q] < values2[p]) or\
                    (values1[q] < values1[p] and values2[q] <= values2[p]):
                n[p] += 1

        # p is ranked by 1
        if n[p] == 0:
            rank[p] = 0
            if p not in front[0]:
                front[0].append(p)

    i = 0
    while front[i] != []:
        # store the solutions with the next rank
        Q = []
        for p in front[i]:
            for q in S[p]:
                n[q] -= 1 # As p is excluded now, decrease n[q]
                if n[q] == 0:
                    rank[q] = i + 1
                    if q not in Q:
                        Q.append(q) # q has the next rank
        i += 1
        front.append(Q)
    del front[-1]
    return front

# 拥挤距离计算
def crowding_distance(values1, values2, front):
    length = len(front)
    distance = [0 for _ in range(length)]
    sorted1 = sorted(range(length), key = lambda idx: values1[idx])
    sorted2 = sorted(range(length), key = lambda idx: values2[idx])
    distance[0] = np.inf
    distance[length - 1] = np.inf
    gap1 = max(values1) - min(values1)
    gap2 = max(values2) - min(values2)
    for idx in range(1, length - 1):
        distance[idx] += (values1[sorted1[idx + 1]] - values2[sorted1[idx - 1]]) / (gap1 + 1e-4) + \
            (values1[sorted2[idx + 1]] - values2[sorted2[idx - 1]]) / (gap2 + 1e-4)
    return distance

In [ ]:
from scipy.linalg import lstsq
import random

def initialize(popSize):
    population = []
    for _ in range(popSize):
        while True:
            ind = np.random.choice((0, 1), p = (1 - k / n, k / n), size = n)
            if ind.sum() <= k:
                break        
        population.append(ind)
    return population

popSize = 20
population = initialize(popSize)

T = round(n*k*k*2*np.exp(1))
mse = []

In [ ]:
function1_values, function2_values = [0 for _ in range(popSize)], [0 for _ in range(popSize)]
function1_values2, function2_values2 = [0 for _ in range(2 * popSize)], [0 for _ in range(2 * popSize)]
for i in range(popSize):
    loss = get_fitness(graph, population[i])
    function2_values[i] = population[i].sum() if (0 < population[i].sum() <= 2 * k) else 999999999999
    function1_values[i] = loss if (0 <= population[i].sum() <= 2 * k) else 999999999999
    
for _ in range(T):
    if _ and _ % (T // 1000) == 0:
        idx = np.array(function2_values) <= k
        cur_mse = min(np.array(function1_values)[idx])
        mse.append(cur_mse)
        print(cur_mse)

    # values
    non_dominated_sorted_solution = fast_non_dominated_sort(function1_values, function2_values)
    population2 = population.copy()

    front_len = len(non_dominated_sorted_solution[0])
    while(len(population2) != 2 * popSize):
        a, b = random.randint(0, front_len - 1), random.randint(0, front_len - 1)
        population2 += crossover(mutation(population[non_dominated_sorted_solution[0][a]].copy()), 
                                 mutation(population[non_dominated_sorted_solution[0][b]].copy()), n)
        
    # values2
    for i in range(2 * popSize):
        loss = get_fitness(graph, population2[i])
        function2_values2[i] = population2[i].sum() if (0 < population2[i].sum() <= 2 * k) else 999999999999
        function1_values2[i] = loss if (0 < population2[i].sum() <= 2 * k) else 999999999999
        
    non_dominated_sorted_solution2 = fast_non_dominated_sort(function1_values2, function2_values2)
    crowding_distance_values2 = [
        crowding_distance(
            function1_values2,
            function2_values2,
            non_dominated_sorted_solution2[i],
        )
        for i in range(len(non_dominated_sorted_solution2))
    ]
    
    new_population = []
    cnt = popSize
    for i in range(len(non_dominated_sorted_solution2)):
        if cnt >= len(non_dominated_sorted_solution2[i]):
            new_population += non_dominated_sorted_solution2[i]
            cnt -= len(non_dominated_sorted_solution2[i])
        else:
            front = sorted(range(len(crowding_distance_values2[i])), key = lambda idx: crowding_distance_values2[i][idx], reverse=True)
            for idx in range(cnt):
                new_population.append(non_dominated_sorted_solution2[i][idx])
            break
    population = [population2[i] for i in new_population]
    for i in range(popSize):
        loss = get_fitness(graph, population[i])
        function2_values[i] = population[i].sum() if (0 < population[i].sum() <= 2 * k) else 999999999999
        function1_values[i] = loss if (0 < population[i].sum() <= 2 * k) else 999999999999

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(function2_values, function1_values)
plt.show()
plt.close()

In [ ]:
print(function1_values, function2_values)

In [ ]:
loss = min(np.array(function1_values))
print(loss)

In [ ]:
# mse = list(filter(lambda x : x != 999999999999, mse))
plt.plot(mse)

In [ ]:
ep = np.array([function1_values, function2_values])
tt = [_ * T // 100 for _ in range(100)]
mse = np.array(mse)


import matplotlib.pyplot as plt
plt.style.use(['science'])
with plt.style.context(['science']):
    plt.figure()
    plt.plot(tt, mse)
    plt.xlabel('epoch')
    plt.ylabel('mse')
    plt.title(f'nsga-ii: {data_path}')
    plt.savefig(f'NSGA-II_{data_path}.pdf')
    plt.close()
    
with plt.style.context(['science']):
    plt.scatter(ep[1], ep[0])
    plt.xlabel('k')
    plt.ylabel('mse')
    plt.title(f'Pareto front- nsga-ii: {data_path}')
    plt.savefig(f'NSGA-II_{data_path}_pareto.pdf', bbox_inches='tight')

In [ ]:
np.save(f'NSGA_{data_path}_t.npy', tt)
np.save(f'NSGA_{data_path}_mse.npy', mse)
np.save(f'NSGA_{data_path}_pareto.npy', ep)